In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from sklearn import tree
from sklearn import ensemble
from sklearn import datasets, linear_model
from sklearn import cross_validation, svm
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import scipy.io
train = scipy.io.loadmat('ReducedImagesForTraining.mat')['images'].T
test = scipy.io.loadmat('ReducedImagesForTesting.mat')['images'].T
print train.shape


def pca(X, wsp):
    n,m = np.shape(X)
    X_mean = np.mean(X, axis = 0)
    X_std = np.std(X,axis = 0)
    X_standarized = (X - X_mean)/X_std
    S = np.cov(X_standarized.T)
    D,V = np.linalg.eigh(S)
    #rint "suma wartosci wlasnych:",np.sum(D)
    Y = np.dot(V.T,X_standarized.T).T
    D_bis = np.matrix(1/np.sqrt(D)).reshape(m,1)
    Y_new = np.multiply(D_bis,Y.T).T
    k = 1; 
    suma = np.sum(D)
    while(np.sum(D[-k:])/suma < wsp):
        k = k+1
    return Y_new[:,-k:], D[-k:], V[:,-k:], k


def PCA(X, k):
    n,m = np.shape(X)
    X_mean = np.mean(X, axis = 0)
    X_std = np.std(X,axis = 0)
    X_standarized = (X - X_mean)/X_std
    S = np.cov(X_standarized.T)
    D,V = np.linalg.eigh(S)
    #print "suma wartosci wlasnych:",np.sum(D)
    Y = np.dot(V.T,X_standarized.T).T
    D_bis = np.matrix(1/np.sqrt(D)).reshape(m,1)
    Y_new = np.multiply(D_bis,Y.T).T
    #print Y_new
    return Y_new[:,-k:]


def nearestPoints(knownData,testData,k):
    D1 = np.sum(np.square(knownData),axis = 1,keepdims=True)
    D2 = np.sum(np.square(testData),axis = 1,keepdims=True).T
    M = D1 + D2 - 2. * np.dot(knownData,testData.T)
    #print M
    return np.asarray(np.argsort(M,axis=0)[:k,:].T)
    

def KNN(knownData, testData, knownClass, testClass,k):
    error = 0
    nearest = nearestPoints(knownData,testData,k)
    #print nearest
    length = testData.shape[0]
    nearestClass = np.empty(length)
    error_list = []
    for i in range(0,length):
        #print i,nearest[i]
        nearestClass[i] = np.argmax(np.bincount([knownClass[a] for a in nearest[i]]))
        if(not(nearestClass[i] == testClass[i])):
           error += 1
           error_list = np.append(error_list, [i])
    return error, error_list

from scipy import misc
from IPython.display import Image
from IPython.display import display
def pokaz(i):
    zdj1 = test[i,:]
    plt.figure(figsize=(10,4))
    plt.imshow(zdj1.reshape(60,82).T, cmap = 'gray')
knownClass = [int((k-1)/5) + 1 for k in range(1,251)]
testClass = [int((k-1)/2) + 1 for k in range(1,101)]
#print knownClass
#print testClass

(250, 4920)


In [2]:
def crossValidation(data,dataClass,k):
    size = data.shape[0]
    arr = np.arange(size)
    np.random.shuffle(arr)
    err = 0
    n = int(size/10)
    for i in range(10):
        mask2 = arr[np.arange(i*n,min((i+1)*n,size))]
        mask1 = arr[np.concatenate((np.arange(i*n),np.arange((i+1)*n,size)))]
        er = KNN(data[mask1],data[mask2],dataClass[mask1],dataClass[mask2],k)
        #print er
        err += er[0]
    return float(err)/(size) * 100

In [3]:
train_new , D, V, k = pca(train, 0.95)
print k
print V.shape
print test.shape
train_new , D, V, k = pca(train, 0.95)
print k
print V.shape
print test.shape
test_mean = np.mean(test, axis = 0)
test_std = np.std(test,axis = 0)
test_standarized = (test - test_mean)/test_std
test_new = np.dot(V.T, test_standarized.T).T
print test_new.shape
test_mean = np.mean(test, axis = 0)
test_std = np.std(test,axis = 0)
test_standarized = (test - test_mean)/test_std
test_new = np.dot(V.T, test_standarized.T).T
print test_new.shape

88
(4920, 88)
(100, 4920)
88
(4920, 88)
(100, 4920)
(100, 88)
(100, 88)


In [4]:
X = np.concatenate((train,test))
print type(X)
#print X.shape
Y = np.asarray(np.concatenate((train_new,test_new)))
print type(Y)
Y2, D, V, k = pca(X,0.95)
Y2 = np.asarray(Y2)
#Y , D1, V1, k1 = pca(train, 0.95)
#print Y.shape
Xclass = np.concatenate((knownClass,testClass))
#print Xclass, Xclass.shape

x1,x2 = zip(*np.random.permutation(zip(X,Xclass)))
X1, X1class = np.asarray(x1),np.asarray(x2)
print X1,X1class
print Xclass
y1,y2 = zip(*np.random.permutation(zip(Y,Xclass)))
Y1, Y1class = np.asarray(y1),np.asarray(y2)
print Y1,Y1class
y11,y12 = zip(*np.random.permutation(zip(Y2,Xclass)))
Y12, Y12class = np.asarray(y11),np.asarray(y12)
print Y12,Y12class

<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[[  48.25         44.58333333   41.08333333 ...,  173.          173.08333333
   171.66666667]
 [  77.           74.08333333   79.16666667 ...,  126.41666667  113.75
    97.33333333]
 [  24.08333333   23.5          24.25       ...,  255.          255.          255.        ]
 ..., 
 [  31.08333333   32.           33.41666667 ...,   91.75         84.           78.5       ]
 [ 255.          255.          255.         ...,  255.          255.          255.        ]
 [  23.91666667   24.41666667   23.25       ...,  255.          215.83333333
    57.5       ]] [16 24  6 40 45  8 37 16  1 48 39  8 45 39  4 32 14 50 31 12 40 13 16 37 24
 24 17 18 44 34 17 15 44 47 26  6 47 19 20 12 25 14 35 38 30 35 50 23  3 22
 20 25 39 36 28 13 11 28  1 29 26 44 31 28 11 43 26 48  7 29 25 21 21 25 46
 25 30 27 37 46 42 29  6 41 10 11 20 20 41 47 14  2 15 21 23 37  1 26 36 45
 13  2 36 13  7 39 22 36 10 40 12 21 20 22 21 39  7 47  7 34 15 41 21 35  9
 46  9 28 17 4

In [5]:
print Y2
print k
#print Y12

[[  5.42848949e-01   1.36095307e+00   1.28075704e+00 ...,   7.39837277e-01
   -1.42111382e-01   1.81469289e-02]
 [ -3.88187595e-01  -1.19048863e-01   1.25604304e+00 ...,   7.86743312e-01
   -1.97893068e-01   4.61754611e-02]
 [ -1.39957018e-01  -5.99382371e-01   1.02351997e+00 ...,   8.34146404e-01
   -1.34783842e-01   5.43702317e-02]
 ..., 
 [ -1.25573344e-01   3.91747628e-01   1.19117969e+00 ...,   2.59235358e-01
   -1.96792734e-03  -1.67526354e+00]
 [ -6.59184041e-02  -2.30256385e+00   4.73693552e-02 ...,   3.34312276e-02
    1.67558951e+00  -1.20032058e+00]
 [ -6.97597652e-03  -1.95054532e+00  -2.08402938e-01 ...,  -2.16016154e-01
   -2.28887345e-01  -2.16060247e+00]]
68


In [6]:
#print KNN(train,test,knownClass,testClass,1)
#print KNN(train_new,test_new,knownClass,testClass,1)
for i in range(1,10):
    print i
    print "Dla oryginalnego zbioru:",crossValidation(X1,X1class,i),"%"
    print "Dla zredukowanych danych:",crossValidation(Y1,Y1class,i),"%"
    print "Dla zredukowanych danych:",crossValidation(Y12,Y12class,i),"%"

1
Dla oryginalnego zbioru: 49.1428571429 %
Dla zredukowanych danych: 32.5714285714 %
Dla zredukowanych danych: 8.57142857143 %
2
Dla oryginalnego zbioru: 57.1428571429 %
Dla zredukowanych danych: 40.8571428571 %
Dla zredukowanych danych: 12.2857142857 %
3
Dla oryginalnego zbioru: 56.2857142857 %
Dla zredukowanych danych: 42.8571428571 %
Dla zredukowanych danych: 12.5714285714 %
4
Dla oryginalnego zbioru: 58.8571428571 %
Dla zredukowanych danych: 38.2857142857 %
Dla zredukowanych danych: 12.0 %
5
Dla oryginalnego zbioru: 63.1428571429 %
Dla zredukowanych danych: 38.5714285714 %
Dla zredukowanych danych: 12.2857142857 %
6
Dla oryginalnego zbioru: 69.1428571429 %
Dla zredukowanych danych: 39.1428571429 %
Dla zredukowanych danych: 13.1428571429 %
7
Dla oryginalnego zbioru: 75.7142857143 %
Dla zredukowanych danych: 41.1428571429 %
Dla zredukowanych danych: 14.2857142857 %
8
Dla oryginalnego zbioru: 74.5714285714 %
Dla zredukowanych danych: 41.4285714286 %
Dla zredukowanych danych: 14.571428

In [7]:
def everything(X,Xclass,k):
    Y = np.asarray(PCA(X,k))
    x1,x2 = zip(*np.random.permutation(zip(X,Xclass)))
    X1, X1class = np.asarray(x1),np.asarray(x2)
    y1,y2 = zip(*np.random.permutation(zip(Y,Xclass)))
    Y1, Y1class = np.asarray(y1),np.asarray(y2)
    for i in range(1,10):
        print i
        print "Dla oryginalnego zbioru:",crossValidation(X1,X1class,i),"%"
        print "Dla zredukowanych danych:",crossValidation(Y1,Y1class,i),"%"

In [8]:
everything(X,Xclass,68)

1
Dla oryginalnego zbioru: 48.5714285714 %
Dla zredukowanych danych: 8.85714285714 %
2
Dla oryginalnego zbioru: 56.5714285714 %
Dla zredukowanych danych: 12.8571428571 %
3
Dla oryginalnego zbioru: 57.7142857143 %
Dla zredukowanych danych: 14.2857142857 %
4
Dla oryginalnego zbioru: 60.0 %
Dla zredukowanych danych: 12.0 %
5
Dla oryginalnego zbioru: 65.7142857143 %
Dla zredukowanych danych: 11.4285714286 %
6
Dla oryginalnego zbioru: 66.0 %
Dla zredukowanych danych: 13.1428571429 %
7
Dla oryginalnego zbioru: 71.7142857143 %
Dla zredukowanych danych: 15.1428571429 %
8
Dla oryginalnego zbioru: 75.1428571429 %
Dla zredukowanych danych: 16.0 %
9
Dla oryginalnego zbioru: 77.4285714286 %
Dla zredukowanych danych: 17.4285714286 %


In [9]:
N = 2000
everything(X,Xclass,N)

1
Dla oryginalnego zbioru: 50.0 %
Dla zredukowanych danych: 98.2857142857 %
2
Dla oryginalnego zbioru: 58.2857142857 %
Dla zredukowanych danych: 98.2857142857 %
3
Dla oryginalnego zbioru: 59.4285714286 %
Dla zredukowanych danych: 98.5714285714 %
4
Dla oryginalnego zbioru: 62.8571428571 %
Dla zredukowanych danych: 98.8571428571 %
5
Dla oryginalnego zbioru: 67.7142857143 %
Dla zredukowanych danych: 98.8571428571 %
6
Dla oryginalnego zbioru: 72.0 %
Dla zredukowanych danych: 99.1428571429 %
7
Dla oryginalnego zbioru: 71.4285714286 %
Dla zredukowanych danych: 98.5714285714 %
8
Dla oryginalnego zbioru: 72.0 %
Dla zredukowanych danych: 98.8571428571 %
9
Dla oryginalnego zbioru: 76.0 %
Dla zredukowanych danych: 99.1428571429 %


In [10]:
N = 1000
everything(X,Xclass,N)

1
Dla oryginalnego zbioru: 48.0 %
Dla zredukowanych danych: 97.4285714286 %
2
Dla oryginalnego zbioru: 57.7142857143 %
Dla zredukowanych danych: 98.0 %
3
Dla oryginalnego zbioru: 56.8571428571 %
Dla zredukowanych danych: 97.7142857143 %
4
Dla oryginalnego zbioru: 60.2857142857 %
Dla zredukowanych danych: 98.5714285714 %
5
Dla oryginalnego zbioru: 67.1428571429 %
Dla zredukowanych danych: 98.8571428571 %
6
Dla oryginalnego zbioru: 66.8571428571 %
Dla zredukowanych danych: 98.5714285714 %
7
Dla oryginalnego zbioru: 70.5714285714 %
Dla zredukowanych danych: 98.5714285714 %
8
Dla oryginalnego zbioru: 77.1428571429 %
Dla zredukowanych danych: 99.1428571429 %
9
Dla oryginalnego zbioru: 77.7142857143 %
Dla zredukowanych danych: 98.5714285714 %


In [11]:
N = 500
everything(X,Xclass,N)

1
Dla oryginalnego zbioru: 49.1428571429 %
Dla zredukowanych danych: 98.0 %
2
Dla oryginalnego zbioru: 59.1428571429 %
Dla zredukowanych danych: 98.0 %
3
Dla oryginalnego zbioru: 58.8571428571 %
Dla zredukowanych danych: 98.0 %
4
Dla oryginalnego zbioru: 58.0 %
Dla zredukowanych danych: 98.2857142857 %
5
Dla oryginalnego zbioru: 64.5714285714 %
Dla zredukowanych danych: 97.7142857143 %
6
Dla oryginalnego zbioru: 67.4285714286 %
Dla zredukowanych danych: 98.0 %
7
Dla oryginalnego zbioru: 71.1428571429 %
Dla zredukowanych danych: 98.2857142857 %
8
Dla oryginalnego zbioru: 73.1428571429 %
Dla zredukowanych danych: 98.8571428571 %
9
Dla oryginalnego zbioru: 80.0 %
Dla zredukowanych danych: 98.2857142857 %


In [12]:
N = 250
everything(X,Xclass,N)

1
Dla oryginalnego zbioru: 47.4285714286 %
Dla zredukowanych danych: 51.4285714286 %
2
Dla oryginalnego zbioru: 58.0 %
Dla zredukowanych danych: 66.8571428571 %
3
Dla oryginalnego zbioru: 57.1428571429 %
Dla zredukowanych danych: 69.1428571429 %
4
Dla oryginalnego zbioru: 61.1428571429 %
Dla zredukowanych danych: 67.4285714286 %
5
Dla oryginalnego zbioru: 63.4285714286 %
Dla zredukowanych danych: 68.5714285714 %
6
Dla oryginalnego zbioru: 66.2857142857 %
Dla zredukowanych danych: 71.1428571429 %
7
Dla oryginalnego zbioru: 68.8571428571 %
Dla zredukowanych danych: 74.2857142857 %
8
Dla oryginalnego zbioru: 72.5714285714 %
Dla zredukowanych danych: 76.2857142857 %
9
Dla oryginalnego zbioru: 76.2857142857 %
Dla zredukowanych danych: 75.1428571429 %


In [13]:
N = 100
everything(X,Xclass,N)

1
Dla oryginalnego zbioru: 48.5714285714 %
Dla zredukowanych danych: 14.5714285714 %
2
Dla oryginalnego zbioru: 58.5714285714 %
Dla zredukowanych danych: 21.4285714286 %
3
Dla oryginalnego zbioru: 59.1428571429 %
Dla zredukowanych danych: 21.4285714286 %
4
Dla oryginalnego zbioru: 60.2857142857 %
Dla zredukowanych danych: 22.2857142857 %
5
Dla oryginalnego zbioru: 62.5714285714 %
Dla zredukowanych danych: 24.8571428571 %
6
Dla oryginalnego zbioru: 68.0 %
Dla zredukowanych danych: 23.4285714286 %
7
Dla oryginalnego zbioru: 69.7142857143 %
Dla zredukowanych danych: 24.0 %
8
Dla oryginalnego zbioru: 75.1428571429 %
Dla zredukowanych danych: 27.7142857143 %
9
Dla oryginalnego zbioru: 78.0 %
Dla zredukowanych danych: 30.0 %


In [14]:
N = 75
everything(X,Xclass,N)

1
Dla oryginalnego zbioru: 49.7142857143 %
Dla zredukowanych danych: 10.5714285714 %
2
Dla oryginalnego zbioru: 58.8571428571 %
Dla zredukowanych danych: 15.1428571429 %
3
Dla oryginalnego zbioru: 57.4285714286 %
Dla zredukowanych danych: 14.5714285714 %
4
Dla oryginalnego zbioru: 60.0 %
Dla zredukowanych danych: 16.2857142857 %
5
Dla oryginalnego zbioru: 64.2857142857 %
Dla zredukowanych danych: 16.0 %
6
Dla oryginalnego zbioru: 66.5714285714 %
Dla zredukowanych danych: 15.1428571429 %
7
Dla oryginalnego zbioru: 72.2857142857 %
Dla zredukowanych danych: 17.4285714286 %
8
Dla oryginalnego zbioru: 77.1428571429 %
Dla zredukowanych danych: 19.7142857143 %
9
Dla oryginalnego zbioru: 76.2857142857 %
Dla zredukowanych danych: 20.2857142857 %


In [15]:
N = 50
everything(X,Xclass,N)

1
Dla oryginalnego zbioru: 47.4285714286 %
Dla zredukowanych danych: 8.57142857143 %
2
Dla oryginalnego zbioru: 57.4285714286 %
Dla zredukowanych danych: 12.2857142857 %
3
Dla oryginalnego zbioru: 55.7142857143 %
Dla zredukowanych danych: 11.4285714286 %
4
Dla oryginalnego zbioru: 60.0 %
Dla zredukowanych danych: 12.2857142857 %
5
Dla oryginalnego zbioru: 66.2857142857 %
Dla zredukowanych danych: 12.0 %
6
Dla oryginalnego zbioru: 68.5714285714 %
Dla zredukowanych danych: 12.5714285714 %
7
Dla oryginalnego zbioru: 71.1428571429 %
Dla zredukowanych danych: 14.0 %
8
Dla oryginalnego zbioru: 74.5714285714 %
Dla zredukowanych danych: 12.8571428571 %
9
Dla oryginalnego zbioru: 79.4285714286 %
Dla zredukowanych danych: 14.5714285714 %


In [16]:
N = 25
everything(X,Xclass,N)

1
Dla oryginalnego zbioru: 48.5714285714 %
Dla zredukowanych danych: 19.7142857143 %
2
Dla oryginalnego zbioru: 58.0 %
Dla zredukowanych danych: 22.8571428571 %
3
Dla oryginalnego zbioru: 56.8571428571 %
Dla zredukowanych danych: 22.5714285714 %
4
Dla oryginalnego zbioru: 61.1428571429 %
Dla zredukowanych danych: 21.4285714286 %
5
Dla oryginalnego zbioru: 63.7142857143 %
Dla zredukowanych danych: 18.0 %
6
Dla oryginalnego zbioru: 65.1428571429 %
Dla zredukowanych danych: 18.8571428571 %
7
Dla oryginalnego zbioru: 72.2857142857 %
Dla zredukowanych danych: 20.0 %
8
Dla oryginalnego zbioru: 74.2857142857 %
Dla zredukowanych danych: 20.8571428571 %
9
Dla oryginalnego zbioru: 76.8571428571 %
Dla zredukowanych danych: 21.7142857143 %


In [17]:
N = 10
everything(X,Xclass,N)

1
Dla oryginalnego zbioru: 47.7142857143 %
Dla zredukowanych danych: 53.7142857143 %
2
Dla oryginalnego zbioru: 55.4285714286 %
Dla zredukowanych danych: 57.7142857143 %
3
Dla oryginalnego zbioru: 58.2857142857 %
Dla zredukowanych danych: 55.4285714286 %
4
Dla oryginalnego zbioru: 61.1428571429 %
Dla zredukowanych danych: 58.8571428571 %
5
Dla oryginalnego zbioru: 64.5714285714 %
Dla zredukowanych danych: 62.2857142857 %
6
Dla oryginalnego zbioru: 66.5714285714 %
Dla zredukowanych danych: 58.0 %
7
Dla oryginalnego zbioru: 70.2857142857 %
Dla zredukowanych danych: 58.8571428571 %
8
Dla oryginalnego zbioru: 74.5714285714 %
Dla zredukowanych danych: 59.4285714286 %
9
Dla oryginalnego zbioru: 76.2857142857 %
Dla zredukowanych danych: 59.7142857143 %


In [18]:
N = 5
everything(X,Xclass,N)

1
Dla oryginalnego zbioru: 48.2857142857 %
Dla zredukowanych danych: 66.8571428571 %
2
Dla oryginalnego zbioru: 56.8571428571 %
Dla zredukowanych danych: 69.7142857143 %
3
Dla oryginalnego zbioru: 56.8571428571 %
Dla zredukowanych danych: 69.7142857143 %
4
Dla oryginalnego zbioru: 61.1428571429 %
Dla zredukowanych danych: 73.1428571429 %
5
Dla oryginalnego zbioru: 65.7142857143 %
Dla zredukowanych danych: 72.0 %
6
Dla oryginalnego zbioru: 69.4285714286 %
Dla zredukowanych danych: 73.7142857143 %
7
Dla oryginalnego zbioru: 72.2857142857 %
Dla zredukowanych danych: 76.2857142857 %
8
Dla oryginalnego zbioru: 74.8571428571 %
Dla zredukowanych danych: 76.2857142857 %
9
Dla oryginalnego zbioru: 77.7142857143 %
Dla zredukowanych danych: 79.7142857143 %


In [19]:
def everything2(X,Xclass,k):
    Y = np.asarray(PCA(X,k))
    X_mean = np.mean(X, axis = 0)
    X_std = np.std(X,axis = 0)
    X_standarized = (X - X_mean)/X_std
    x1,x2 = zip(*np.random.permutation(zip(X_standarized,Xclass)))
    X1, X1class = np.asarray(x1),np.asarray(x2)
    y1,y2 = zip(*np.random.permutation(zip(Y,Xclass)))
    Y1, Y1class = np.asarray(y1),np.asarray(y2)
    for i in range(1,10):
        print i
        print "Dla oryginalnego zbioru:",crossValidation(X1,X1class,i),"%"
        print "Dla zredukowanych danych:",crossValidation(Y1,Y1class,i),"%"

In [20]:
N = 68
everything2(X,Xclass,N)
#standaryzacja specjalnie nie pomogla

1
Dla oryginalnego zbioru: 50.0 %
Dla zredukowanych danych: 9.71428571429 %
2
Dla oryginalnego zbioru: 56.8571428571 %
Dla zredukowanych danych: 12.2857142857 %
3
Dla oryginalnego zbioru: 52.8571428571 %
Dla zredukowanych danych: 12.5714285714 %
4
Dla oryginalnego zbioru: 59.1428571429 %
Dla zredukowanych danych: 11.1428571429 %
5
Dla oryginalnego zbioru: 61.7142857143 %
Dla zredukowanych danych: 12.5714285714 %
6
Dla oryginalnego zbioru: 63.1428571429 %
Dla zredukowanych danych: 14.5714285714 %
7
Dla oryginalnego zbioru: 67.4285714286 %
Dla zredukowanych danych: 14.5714285714 %
8
Dla oryginalnego zbioru: 71.7142857143 %
Dla zredukowanych danych: 15.7142857143 %
9
Dla oryginalnego zbioru: 71.4285714286 %
Dla zredukowanych danych: 17.1428571429 %
